<a href="https://colab.research.google.com/github/biraj21/deep_learning_with_tf/blob/main/ch4/newswires_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import reuters

In [2]:
def vectorize_sequences(sequences, dims=10_000):
    result = np.zeros((len(sequences), dims))
    for i, sequence in enumerate(sequences):
        for word_index in sequence:
            result[i, word_index] = 1

    return result

In [3]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10_000)
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)

In [4]:
model = keras.Sequential([
    layers.Dense(128, activation="relu"),
    layers.Dense(46, activation="softmax"),
])

model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    x_train, y_train,
    epochs=6,
    batch_size=256
)

Epoch 1/6
36/36 [==============================] - 2s 52ms/step - loss: 1.7992 - accuracy: 0.6552
Epoch 2/6
36/36 [==============================] - 2s 69ms/step - loss: 0.8601 - accuracy: 0.8244
Epoch 3/6
36/36 [==============================] - 2s 56ms/step - loss: 0.5649 - accuracy: 0.8847
Epoch 4/6
36/36 [==============================] - 1s 34ms/step - loss: 0.3970 - accuracy: 0.9181
Epoch 5/6
36/36 [==============================] - 1s 34ms/step - loss: 0.2955 - accuracy: 0.9338
Epoch 6/6
36/36 [==============================] - 1s 34ms/step - loss: 0.2334 - accuracy: 0.9443


In [5]:
model.evaluate(x_test, y_test)

71/71 [==============================] - 0s 3ms/step - loss: 0.8848 - accuracy: 0.8014


[0.8847751617431641, 0.8014247417449951]